# Carga de datos

En el apartado anterior se ha explicado los pasos que se han realizado para normalizar las reseñas de películas y TV de Amazon. Para ello, lo primero que se ha hecho ha sido descargar los datos presentes en la página web y luego se ha ido realizando distintas modificaciones a los datos para poder prepararlos para generar un modelo de clasificación binaria. Durante el preprocesamiento, se han ido generando distintos ficheros CSV con la finalidad de tener puntos de control a los cuales poder acceder en caso de que se necesitaban. Concretamente, en el apartado anterior, una vez que se han descargado los datos se ha realizado un punto de control. Este punto de control son dos ficheros CSV los cuales contienen por un lado los datos de entrenamiento y por otro lado los datos de test. Dado que el proceso de descargar de nuevo los datos desde la página web es un proceso que lleva su tiempo, este apartado se va a comenzar desde el punto de control comentado. Destacar que los datos presentes en estos ficheros son datos en bruto y no se les ha aplicado ninguna normalización.

Por ello, lo primero que se va a hacer es conectarse al google drive para poder importar los ficheros.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Seguidamente importo los datos de entrenamiento y de test.

In [ ]:
#Datos de entrenamiento
import pandas as pd
train_data = pd.read_csv("/content/drive/MyDrive/Práctica NPL/data_train.csv")
train_data = train_data.dropna(subset=['review_text'])
print(train_data.head())
print("Dimensión conjunto de entrenamiento",train_data.shape)

                                         review_text  rating_note
0  I loathe most American detective dramas. There...          5.0
1  The storyline was great.  Looking for the last...          3.0
2  Well, at least the first film was bearable. It...          3.0
3  Whilst not being near the best of Crosby's fil...          5.0
4  Best season so far. The cast has outdone thems...          5.0
Dimensión conjunto de entrenamiento (99164, 2)


In [ ]:
#Datos de test
import pandas as pd
test_data = pd.read_csv("/content/drive/MyDrive/Práctica NPL/data_test.csv")
test_data = test_data.dropna(subset=['review_text'])
print(test_data.head())
print("Dimensión conjunto de test",test_data.shape)

                                         review_text  rating_note
0  I cannot say this strongly enough: this is one...          5.0
1  I wonder if these guys were hoping I would be ...          1.0
2  this movie was boring, i mean real boring, don...          1.0
3  Was an interesting movie with good acting and ...          3.0
4  Kids liked this movie probably better than the...          4.0
Dimensión conjunto de test (24790, 2)


#Pipeline train

Según la definicón actual, en el contexto del Procesamiento del Lenguaje Natural (NLP), un pipeline se refiere a una **secuencia ordenada de pasos** o procesos que **se aplican a un texto o corpus de texto** para realizar una serie de **tareas de procesamiento, análisis o extracción de información**. Estos pasos pueden incluir desde tareas básicas como tokenización y limpieza de texto hasta tareas más complejas como análisis sintáctico, extracción de entidades nombradas, clasificación de sentimientos, entre otros.

Como puede apreciarse, en negrita, se ha señalado los conceptos más importantes de la definición.


*   Secuencia ordenada de pasos: Se refiere a un proceso ordenado en el que se ejecutan una serie de pasos.
*   Se aplican a un texto o corpus de texto: Estos pasos se deben aplicar al corpus con el que se este trabajando, en este caso, las reseñas de Amazon.
*   Tareas de procesamiento, análisis o extracción de información: Con lo que se busca con el pipeline es realizar el preprocesamiento y análisis del corpus.

Por lo tanto, por medio del pipeline lo que se pretende es conseguir realizar todo el preprocesamiento de datos descrito en el apartado 1 de la práctica de forma ordenada y paso por paso, con el fin de conseguir el mismo resultado. Para ello, lo primero que se debe pensar es cuales son los pasos que se han realiado en el apartado anterior.



En el apartado anterior, lo primero que se ha hecho ha sido explorar la distribución de los datos. Sin embargo, la conclusión de esta exploración ha sido que los datos no estaban distribuidos equitativamente pero que no se iba aplicar ninguna técnica para cambiar esta distribución debido a que se pretendía asemejar este problema a un problema real. Por lo tanto, dado que al final no se ha hecho nada, en el pipeline no es necesario incluir esta sección.

A continuación, se ha procedido a realizar la normalización del corpus. Los cambios que se han realizado sobre el corpus han sido:

*   Contracciones: Previo a tokenizar las palabras, primero se han expandido las contracciones dado que las reseñas están escritas en inglés.
*   Tokenización: Se ha divido cada reseña en tokens. El resultado se ha almacenado en la columna review_tokens.
*   Eliminación de stopwords, signos de puntuación y mayúsculas: El resultado se ha almacenado en la columna clean_tokens.
*   Comprobación de símbolos y eliminación de los símbolos ["``", "''", "...", "--", "..", "..."]: Dado que eran tokens que se repetían mucho y que no aportaban información, se han eliminado. El resultado se ha almacenado en la columna clean_tokens.
*   Lematización del vocabulario: Se han lematizado todos los tokens que eran verbos y se han almacenados en la columna lemmatized_tokens.

Una vez que se había conseguido normalizar las reseñas, se ha procedido a analizar la cardinalidad del vocabulario y de cada reseña. Se ha realizado un análisis y como consecuencia de este estudio, se han eliminado todas aquellas reseñas que poseían más de 400 tokens ya normalizados debido a que se han considerado outliers.

Seguidamente, se ha realizado un estudio sobre los datos con los que se trabaja y para ello se han analizado los unigramas, bigramas, trigramas y nube de palabras. Destacar que simplemente se ha analizado y no se han modificado los datos.

Por último, se han generado los embeddings para cada reseña y se han almacenado todos ellos en un tensor resultante de dimensión (98088, 62, 100), siendo 62 el valor del tercer cuartil de los datos y 100 el número de características para cada palabra. Con estos embeddings, más adelante, se han graficado las palabras más similares a una dada.

Por lo tanto, este pipeline tiene que realizar los mismos pasos que se han realiado en el apartado anterior pero solamente aquellos en los que se han modificado los datos. La entrada al pipeline tienen que ser los datos en bruto y la salida tiene que ser el tensor de embeddings. Todos los análisis, gráficas, n-grams, nube de palabras no se deben realizar. Por ello, a continuación procedo a su implementación.

In [ ]:
pip install contractions

In [ ]:
import pandas as pd
import re
import contractions
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import string

# Descargo los recursos necesarios de NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')  # Nuevo

# Inicializo lematizador
lemmatizer = WordNetLemmatizer()

# Símbolos a eliminar
symbols_to_remove = ["``", "''", "...", "--", "..", "..."]

# Conjunto de stopwords y puntuación
stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation)

# Función para expandir contracciones y quitar posesiones
def clean_text(text):
    expanded_text = contractions.fix(text)
    text_without_possession = expanded_text.replace("'s", "")
    return text_without_possession

# Función para limpiar los tokens
def clean_tokens(tokens):
    clean_tokens = [token.lower() for token in tokens if token.lower() not in stop_words and token not in punctuation]
    return clean_tokens

# Función para eliminar símbolos
def remove_symbols(tokens):
    clean_tokens = [re.sub('|'.join(map(re.escape, symbols_to_remove)), '', token) for token in tokens]
    clean_tokens = [token.strip() for token in clean_tokens if token.strip()]
    return clean_tokens

# Función para lemmatizar los tokens
def lemmatize_tokens(tokens):
    lemmatized_tokens = []
    for word in tokens:
        pos_tag = nltk.pos_tag([word])[0][1][0].upper()
        wordnet_pos = {
            'N': wordnet.NOUN,
            'V': wordnet.VERB,
            'J': wordnet.ADJ,
            'R': wordnet.ADV
        }.get(pos_tag, wordnet.NOUN)
        lemmatized_word = lemmatizer.lemmatize(word, pos=wordnet_pos).lower().strip()
        lemmatized_tokens.append(lemmatized_word)
    return lemmatized_tokens

# Crear pipeline
def nlp_pipeline(text):
    # Expandir contracciones y quitar posesiones
    expanded_text = clean_text(text)

    # Tokenizar el texto
    tokens = word_tokenize(expanded_text)

    # Limpiar los tokens
    cleaned_tokens = clean_tokens(tokens)

    # Eliminar símbolos
    symbols_removed = remove_symbols(cleaned_tokens)

    # Lematizar los tokens
    lemmatized_tokens = lemmatize_tokens(symbols_removed)

    return lemmatized_tokens

# Aplico el pipeline, elimino la columna review_text y reordeno el dataser
train_data['tokens'] = train_data['review_text'].apply(nlp_pipeline)
train_data['cardinality'] = train_data['tokens'].apply(len)
train_data.drop(columns=['review_text'], inplace=True)
train_data = train_data[['tokens', 'cardinality', 'rating_note']]

# Por último elimino las filas donde review_cardinality sea superior a 400
train_data = train_data[train_data['cardinality'] <= 400]

# Imprimio la dimensión del dataset para comprobar que todo se ha aplicado correctamente
print("Dimensión del dataset:", train_data.shape)
print(train_data.head())


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Dimensión del dataset: (98088, 3)
                                              tokens  cardinality  rating_note
0  [loathe, american, detective, drama, ending, a...          133          5.0
1  [storyline, great, look, last, chapter, love, ...           68          3.0
2  [well, least, first, film, bearable, glorious,...           24          3.0
3  [whilst, near, best, crosby, film, welcome, se...           29          5.0
4  [best, season, far, cast, outdone, kathy, bate...           12          5.0


En el primer apartado una vez que había normalizado los datos, he procedido a generar los embeddings. Sin embargo, en este caso no voy a generarlos dado que en el apartado 3 se solicita generar dos modelos. En mi caso, voy a utilizar un modelo SVM y un modelo LSTM. Dado que el primer modelo no nececesita embeddings, no creo que sea necesario crear los embeddings en este apartado, sino que creo que es más conveniente crearlos en el momento de creación del modelo LSTM. Por ello, lo que voy a hacer es guardar el resultado obtenido en un fichero CSV para poder importarlo en el apartado 3.

In [ ]:
import pandas as pd

# Ruta donde guardar el archivo CSV
ruta_archivo = "/content/drive/MyDrive/Práctica NPL/data_train_pipeline.csv"

# Guardo el DataFrame en un archivo CSV
train_data.to_csv(ruta_archivo, index=False, sep=';')


#Pipeline test

Por otro lado, la normalización se ha realizado solamente para los datos de train. Es importante aplicar los mismos cambios a test para que así se pueda generar los modelos y que estos puedan aprender. Lo bueno de haber definido el pipeline es que es muy sencillo aplicar los cambios ya que simplemente es ejecutarlo pero en este caso a test.

In [ ]:
import pandas as pd
import re
import contractions
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

# Descargo los recursos necesarios de NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')  # Nuevo

# Inicializo lematizador
lemmatizer = WordNetLemmatizer()

# Símbolos a eliminar
symbols_to_remove = ["``", "''", "...", "--", "..", "..."]

# Conjunto de stopwords y puntuación
stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation)

# Función para expandir contracciones y quitar posesiones
def clean_text(text):
    expanded_text = contractions.fix(text)
    text_without_possession = expanded_text.replace("'s", "")
    return text_without_possession

# Función para limpiar los tokens
def clean_tokens(tokens):
    clean_tokens = [token.lower() for token in tokens if token.lower() not in stop_words and token not in punctuation]
    return clean_tokens

# Función para eliminar símbolos
def remove_symbols(tokens):
    clean_tokens = [re.sub('|'.join(map(re.escape, symbols_to_remove)), '', token) for token in tokens]
    clean_tokens = [token.strip() for token in clean_tokens if token.strip()]
    return clean_tokens

# Función para lemmatizar los tokens
def lemmatize_tokens(tokens):
    lemmatized_tokens = []
    for word in tokens:
        pos_tag = nltk.pos_tag([word])[0][1][0].upper()
        wordnet_pos = {
            'N': wordnet.NOUN,
            'V': wordnet.VERB,
            'J': wordnet.ADJ,
            'R': wordnet.ADV
        }.get(pos_tag, wordnet.NOUN)
        lemmatized_word = lemmatizer.lemmatize(word, pos=wordnet_pos).lower().strip()
        lemmatized_tokens.append(lemmatized_word)
    return lemmatized_tokens

# Crear pipeline
def nlp_pipeline(text):
    # Expandir contracciones y quitar posesiones
    expanded_text = clean_text(text)

    # Tokenizar el texto
    tokens = word_tokenize(expanded_text)

    # Limpiar los tokens
    cleaned_tokens = clean_tokens(tokens)

    # Eliminar símbolos
    symbols_removed = remove_symbols(cleaned_tokens)

    # Lematizar los tokens
    lemmatized_tokens = lemmatize_tokens(symbols_removed)

    return lemmatized_tokens

# Aplico el pipeline, elimino la columna review_text y reordeno el dataser
test_data['tokens'] = test_data['review_text'].apply(nlp_pipeline)
test_data['cardinality'] = test_data['tokens'].apply(len)
test_data.drop(columns=['review_text'], inplace=True)
test_data = test_data[['tokens', 'cardinality', 'rating_note']]

# Por último elimino las filas donde review_cardinality sea superior a 400
test_data = test_data[test_data['cardinality'] <= 400] #IMPORTANTE: EL umbral para considerar outliers debe ser el mimso que el de train, sino se estaría cometiendo contaminación.

# Imprimio la dimensión del dataset para comprobar que todo se ha aplicado correctamente
print("Dimensión del dataset:", test_data.shape)
print(test_data.head())


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Dimensión del dataset: (24543, 3)
                                              tokens  cardinality  rating_note
0  [say, strongly, enough, one, man, personal, vi...           21          5.0
1  [wonder, guy, hop, would, influence, something...           98          1.0
2  [movie, boring, mean, real, boring, rent, wish...           17          1.0
3  [interest, movie, good, act, reasonable, plot,...           14          3.0
4  [kid, like, movie, probably, well, first, one,...           14          4.0


Guardo los datos de test ya normalizados.

In [ ]:
import pandas as pd

# Ruta donde guardar el archivo CSV
ruta_archivo = "/content/drive/MyDrive/Práctica NPL/data_test_pipeline.csv"

# Guardo el DataFrame en un archivo CSV
test_data.to_csv(ruta_archivo, index=False, sep=';')


#Conclusión apartado 2


Al igual que en el apartado 1, recapitulo en esta sección que es lo que se ha hecho para no perder el contexto de la práctica. En este apartado lo que se ha realizado es una función que permite procesar las reseñas y prepararlas para que un modelo pueda ser entrenado. Para ello, cada reseña se ha normalizado y tokenizado en base a los mismos pasos indicados en el apartado 1 y como conseuencia se han generados dos datasets distintos: uno para train y otro para test. Ambos dos, se han almacenado en ficheros csv para que luego puedan ser utilizados en el siguiente apartado.